In [1]:
import json
from pathlib import Path

json_path = Path("config/json/symbols.json")
json_path.parent.mkdir(parents=True, exist_ok=True)

symbolos_test = {
    "simbolos": ["AAPL", "TSLA", "AMD", "NVDA", "MSFT", "SMCI", "XOM"]
}

with open(json_path, "w") as f:
    json.dump(symbolos_test, f, indent=4)

print(f"Archivo de prueba creado en: {json_path.resolve()}")

Archivo de prueba creado en: C:\Users\leant\OneDrive\Documentos\trading\Jupyter\notebooks\test\config\json\symbols.json


In [2]:
# Conteo de simbolos presentes en el historico y generacion de symbols.json

import json
from pathlib import Path

# === Rutas
BASE_PATH = Path("..").resolve().parent
HISTORIC_DIR = BASE_PATH / "data" / "historic"
CONFIG_PATH = Path("config/json/symbols.json")
CONFIG_PATH.parent.mkdir(parents=True, exist_ok=True)

# === Extraer simbolos desde nombres de archivo
symbolos_detectados = sorted([f.stem.upper() for f in HISTORIC_DIR.glob("*.parquet")])

# === Guardar JSON
with open(CONFIG_PATH, "w") as f:
    json.dump({"simbolos": symbolos_detectados}, f, indent=4)

print(f"Archivo symbols.json generado con {len(symbolos_detectados)} simbolos:")
print(symbolos_detectados)


Archivo symbols.json generado con 673 simbolos:
['A', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACI', 'ACM', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AFG', 'AFL', 'AFRM', 'AGNCL', 'AGNCM', 'AGNCN', 'AGNCO', 'AGNCP', 'AIG', 'AJG', 'AKAM', 'ALAB', 'ALGN', 'ALL', 'ALLE', 'ALLY', 'ALNY', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMH', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'AON', 'APD', 'APG', 'APH', 'APO', 'APOS', 'APP', 'AQNB', 'AR', 'ARCC', 'ARE', 'ARES', 'ARMK', 'ATI', 'ATO', 'ATR', 'AUR', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXON', 'AXP', 'AZO', 'BA', 'BAC', 'BAH', 'BALL', 'BAX', 'BBY', 'BDX', 'BEN', 'BEPC', 'BG', 'BIIB', 'BJ', 'BK', 'BKNG', 'BKR', 'BLDR', 'BLK', 'BMRN', 'BMY', 'BR', 'BRO', 'BROS', 'BSX', 'BSY', 'BTSGU', 'BURL', 'BX', 'BXP', 'C', 'CACI', 'CAG', 'CAH', 'CARR', 'CART', 'CASY', 'CAT', 'CBRE', 'CCI', 'CCK', 'CCL', 'CCZ', 'CDNS', 'CDW', 'CEG', 'CF', 'CFG', 'CG', 'CHD', 'CHRW', 'CHTR', 'CHWY', 'CI', 'CIEN', 'CINF', 'CL', 'CLH', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNA', 'CNC', 'CNM',

In [3]:
!python ../../scripts/core/gen_senales_parallel.py

Simbolos a procesar: 673
Iniciando procesamiento paralelo con debug...
ABNB: ejecutando 15 estrategias...
ABNB: breakout_volumen -> 89 senales
ABNB: cruce_medias -> 38 senales
ABNB: ema_9_21_cruce -> 52 senales
ABNB: ema_pullback -> sin senales
ABNB: gap_open_strategy -> 74 senales
ABNB: inside_bar_breakout -> sin senales
ABNB: ma_envelope_reversals -> 678 senales
ABNB: macd_cruce -> 95 senales
ABNB: macd_hist_reversal -> 614 senales
ABNB: mov_avg -> 63 senales
ABNB: rsi_divergencia -> 121 senales
ABNB: rsi_reversion -> 250 senales
ABNB: sma_10_50_cruce -> 30 senales
ABNB: soporte_resistencia -> 189 senales
ABNB: volatilidad_contraria -> sin senales

ABBV: ejecutando 15 estrategias...
ABBV: breakout_volumen -> 260 senales
ABBV: cruce_medias -> 111 senales
ABBV: ema_9_21_cruce -> 129 senales
ABBV: ema_pullback -> sin senales
ABBV: gap_open_strategy -> 48 senales
ABBV: inside_bar_breakout -> sin senales
ABBV: ma_envelope_reversals -> 1268 senales
ABBV: macd_cruce -> 239 senales
ABBV: mac

In [4]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Rutas
BASE_PATH = Path("..").resolve().parent
REPORTS_DIR = BASE_PATH / "reports" / "senales_heuristicas"
FECHA = date.today().isoformat()
ARCHIVO = REPORTS_DIR / f"senales_heuristicas_{FECHA}.csv"

# === Leer archivo
df = pd.read_csv(ARCHIVO)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

print(f"Total de señales: {len(df)}")
display(df.head(5))

# === Verificación de estructura
print("\nColumnas:", list(df.columns))
print("Fechas:", df["fecha"].min().date(), "-", df["fecha"].max().date())
print("Estrategias únicas:", df["estrategia"].nunique())
print("Símbolos únicos:", df["simbolo"].nunique())


# === Verificación de estructura
print("\nColumnas:", list(df.columns))
print("Fechas:", df["fecha"].min().date(), "→", df["fecha"].max().date())
print("Estrategias únicas:", df["estrategia"].nunique())
print("Símbolos únicos:", df["simbolo"].nunique())

# === Métricas por estrategia
resumen_estrategia = (
    df.groupby("estrategia")["signal"]
    .value_counts()
    .unstack(fill_value=0)
    .assign(total=lambda d: d.sum(axis=1))
    .sort_values("total", ascending=False)
)

print("\nResumen por estrategia:")
display(resumen_estrategia.head(10))

# === Señales por símbolo
resumen_simbolo = (
    df.groupby("simbolo")["signal"]
    .value_counts()
    .unstack(fill_value=0)
    .assign(total=lambda d: d.sum(axis=1))
    .sort_values("total", ascending=False)
)

print("\nResumen por símbolo:")
display(resumen_simbolo.head(10))

# === Señales por día (actividad temporal)
actividad_diaria = df.groupby("fecha").size()
print("\nActividad por fecha:")
display(actividad_diaria.tail(10))

# === Señales por combinación estrategia + símbolo (top 10)
combinaciones_top = (
    df.groupby(["estrategia", "simbolo"]).size()
    .sort_values(ascending=False)
    .head(20)
    .reset_index(name="total")
)

print("\nTop combinaciones estrategia + símbolo:")
display(combinaciones_top)


Total de señales: 5134161


,fecha,simbolo,estrategia,signal
0,2021-01-14,ABNB,breakout_volumen,buy
1,2021-01-27,ABNB,breakout_volumen,buy
2,2021-05-06,ABNB,breakout_volumen,sell
3,2021-05-10,ABNB,breakout_volumen,sell
4,2021-05-11,ABNB,breakout_volumen,sell



Columnas: ['fecha', 'simbolo', 'estrategia', 'signal']
Fechas: 1998-06-08 - 2025-05-23
Estrategias únicas: 13
Símbolos únicos: 673

Columnas: ['fecha', 'simbolo', 'estrategia', 'signal']
Fechas: 1998-06-08 → 2025-05-23
Estrategias únicas: 13
Símbolos únicos: 673

Resumen por estrategia:


signal,buy,sell,total
estrategia,,,
macd_hist_reversal,726648,711234,1437882
ma_envelope_reversals,511443,683698,1195141
rsi_reversion,257828,430796,688624
soporte_resistencia,284861,200393,485254
rsi_divergencia,127113,209020,336133
breakout_volumen,123666,97680,221346
macd_cruce,109232,109291,218523
mov_avg,85852,85733,171585
ema_9_21_cruce,61379,61236,122615



Resumen por símbolo:


signal,buy,sell,total
simbolo,,,
AMD,5676,5729,11405
FCX,5452,5763,11215
EXAS,5347,5735,11082
URI,4967,6044,11011
INSM,5554,5408,10962
THC,5347,5601,10948
NVDA,4966,5966,10932
MU,5246,5686,10932
GME,5580,5349,10929



Actividad por fecha:


fecha
2025-05-12    2544
2025-05-13    2028
2025-05-14    1715
2025-05-15    1458
2025-05-16    1545
2025-05-19    1426
2025-05-20    1278
2025-05-21    1077
2025-05-22    1064
2025-05-23    1154
dtype: int64


Top combinaciones estrategia + símbolo:


,estrategia,simbolo,total
0,ma_envelope_reversals,AMD,3527
1,ma_envelope_reversals,INSM,3520
2,ma_envelope_reversals,EXAS,3509
3,ma_envelope_reversals,BLDR,3373
4,ma_envelope_reversals,THC,3359
5,ma_envelope_reversals,GME,3352
6,ma_envelope_reversals,MU,3328
7,ma_envelope_reversals,ATI,3310
8,ma_envelope_reversals,ALNY,3310
9,ma_envelope_reversals,UAL,3282


In [5]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Rutas
BASE_PATH = Path("..").resolve().parent
REPORTS_DIR = BASE_PATH / "reports" / "senales_heuristicas"
FECHA = date.today().isoformat()
ARCHIVO = REPORTS_DIR / f"senales_heuristicas_{FECHA}.csv"

# === Leer CSV
df = pd.read_csv(ARCHIVO)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

# === Verificaciones clave
print(f"Total filas: {len(df)}")
print("Tipos de columnas:")
print(df.dtypes)

# === Verificar valores unicos
print("\nValores unicos en 'signal':", df["signal"].unique())
print("Valores unicos en 'fecha':", df['fecha'].nunique())
print("Fecha minima:", df["fecha"].min())
print("Fecha maxima:", df["fecha"].max())

# === Señales por tipo
conteo = df["signal"].value_counts()
print("\nConteo por tipo de señal:")
print(conteo)

# === Detectar valores vacíos
faltantes = df.isna().sum()
print("\nColumnas con valores nulos:")
print(faltantes[faltantes > 0])


Total filas: 5134161
Tipos de columnas:
fecha         datetime64[ns]
simbolo               object
estrategia            object
signal                object
dtype: object

Valores unicos en 'signal': ['buy' 'sell']
Valores unicos en 'fecha': 7155
Fecha minima: 1998-06-08 00:00:00
Fecha maxima: 2025-05-23 00:00:00

Conteo por tipo de señal:
sell    2710909
buy     2423252
Name: signal, dtype: int64

Columnas con valores nulos:
Series([], dtype: int64)


In [6]:
import pandas as pd
from pathlib import Path

# === Ruta al archivo Parquet a verificar
ARCHIVO = Path("..") / ".." / "data" / "historic" / "aapl.parquet"  # Cambia el nombre de archivo si lo necesitas

# === Cargar archivo
df = pd.read_parquet(ARCHIVO)

# === Diagnóstico de columnas e índice
print(f"Columnas disponibles: {list(df.columns)}")
print(f"Tipo de índice: {type(df.index)}")
print(f"¿Índice es datetime? -> {pd.api.types.is_datetime64_any_dtype(df.index)}")

# === Mostrar primeras filas
display(df.head(3))


Columnas disponibles: ['datetime', 'open', 'high', 'low', 'close', 'volume']
Tipo de índice: <class 'pandas.core.indexes.range.RangeIndex'>
¿Índice es datetime? -> False


,datetime,open,high,low,close,volume
0,2005-07-12,1.36536,1.37143,1.35393,1.36571,387038400
1,2005-07-13,1.36750,1.37500,1.35357,1.36964,684835200
2,2005-07-14,1.45679,1.50036,1.43679,1.45536,2096060400


In [7]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Ruta del archivo generado
BASE_PATH = Path("..").resolve().parent
FECHA = date.today().isoformat()
ARCHIVO = BASE_PATH / "reports" / "senales_heuristicas" / f"senales_heuristicas_{FECHA}.csv"

# === Cargar archivo
df = pd.read_csv(ARCHIVO)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

# === Filtrar solo registros con fecha valida
df_validas = df[df["fecha"].notna()]

# === Total simbolos con al menos una senal valida
simbolos_con_senales = df_validas["simbolo"].nunique()
simbolos_total = df["simbolo"].nunique()

print(f"Simbolos con senales validas: {simbolos_con_senales} / {simbolos_total}")

# === Rango de fechas por simbolo
rango_fechas = (
    df_validas.groupby("simbolo")["fecha"]
    .agg(["min", "max"])
    .reset_index()
    .sort_values("simbolo")
)

print("\nRango de fechas por simbolo:")
display(rango_fechas)


Simbolos con senales validas: 673 / 673

Rango de fechas por simbolo:


,simbolo,min,max
0,A,2005-07-14,2025-05-20
1,AAPL,2005-07-13,2025-05-23
2,ABBV,2013-01-03,2025-05-22
3,ABNB,2020-12-11,2025-05-23
4,ABT,2005-07-13,2025-05-23
...,...,...,...
668,ZBRA,2005-07-13,2025-05-23
669,ZG,2011-07-21,2025-05-23
670,ZM,2019-04-22,2025-05-23
671,ZS,2018-03-19,2025-05-23


In [8]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Ruta del archivo generado
BASE_PATH = Path("..").resolve().parent
FECHA = date.today().isoformat()
ARCHIVO = BASE_PATH / "reports" / "senales_heuristicas" / f"senales_heuristicas_{FECHA}.csv"

# === Cargar archivo
df = pd.read_csv(ARCHIVO)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

# === Filtrar registros con fecha valida
df_validas = df[df["fecha"].notna()]

# === Agrupar por estrategia
resumen = (
    df_validas.groupby("estrategia")
    .agg(
        total_signals=("signal", "count"),
        buy_signals=("signal", lambda x: (x == "buy").sum()),
        sell_signals=("signal", lambda x: (x == "sell").sum()),
        fecha_min=("fecha", "min"),
        fecha_max=("fecha", "max")
    )
    .reset_index()
    .sort_values("total_signals", ascending=False)
)

# === Mostrar resultado
print("Resumen por estrategia:")
display(resumen)


Resumen por estrategia:


,estrategia,total_signals,buy_signals,sell_signals,fecha_min,fecha_max
6,macd_hist_reversal,1437882,726648,711234,1998-06-08,2025-05-23
4,ma_envelope_reversals,1195141,511443,683698,1998-07-07,2025-05-23
9,rsi_reversion,688624,257828,430796,1998-08-05,2025-05-23
11,soporte_resistencia,485254,284861,200393,1998-07-22,2025-05-23
8,rsi_divergencia,336133,127113,209020,1998-08-06,2025-05-23
0,breakout_volumen,221346,123666,97680,1998-08-03,2025-05-23
5,macd_cruce,218523,109232,109291,1998-06-08,2025-05-23
7,mov_avg,171585,85852,85733,1998-07-21,2025-05-23
2,ema_9_21_cruce,122615,61379,61236,1998-06-08,2025-05-23
1,cruce_medias,96890,48513,48377,1998-06-08,2025-05-23


In [9]:
import pandas as pd
from pathlib import Path
from datetime import datetime

# === Rutas
BASE_PATH = Path("..").resolve().parent
FECHA_HORA = datetime.now().strftime("%Y-%m-%d_%H-%M")
ARCHIVO = BASE_PATH / "reports" / "senales_heuristicas" / f"senales_heuristicas_{datetime.today().date()}.csv"
LOG_FILE = BASE_PATH / "reports" / "senales_heuristicas" / f"diagnostico_senales_{FECHA_HORA}.txt"

# === Leer archivo
df = pd.read_csv(ARCHIVO)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

# === Diagnostico
output = []

output.append(f"Verificacion del archivo: {ARCHIVO.name}")
output.append(f"Total filas: {len(df)}")

# Columnas necesarias
columnas_necesarias = {"fecha", "simbolo", "estrategia", "signal"}
columnas_presentes = set(df.columns)
faltantes = columnas_necesarias - columnas_presentes

output.append("\nColumnas requeridas: " + str(columnas_necesarias))
output.append("Columnas presentes : " + str(columnas_presentes))
if faltantes:
    output.append(f"❌ Faltan columnas: {faltantes}")
else:
    output.append("✅ Todas las columnas requeridas están presentes.")

# Tipos de datos
output.append("\nTipos de columnas:")
output.append(str(df.dtypes))

# Señales
output.append("\nValores únicos en 'signal': " + str(df["signal"].unique().tolist()))
conteo = df["signal"].value_counts()
output.append("\nConteo por tipo de señal:")
output.append(str(conteo))

# Rango de fechas
output.append("\nFechas:")
output.append("Fecha mínima: " + str(df["fecha"].min()))
output.append("Fecha máxima: " + str(df["fecha"].max()))

# Nulos
nulos = df.isna().sum()
nulos_presentes = nulos[nulos > 0]
output.append("\nColumnas con valores nulos:")
output.append(str(nulos_presentes if not nulos_presentes.empty else "Sin nulos"))

# Veredicto
if not faltantes and df["fecha"].notna().all() and df["signal"].isin(["buy", "sell"]).all():
    output.append("\n✅ El archivo de señales esta listo para backtesting.")
else:
    output.append("\n⚠️ El archivo de señales tiene problemas que deben resolverse antes del backtesting.")

# === Imprimir resumen en pantalla
print("\n".join(output))

# === Guardar log con marca de tiempo
with open(LOG_FILE, "w", encoding="utf-8") as f:
    f.write("\n".join(output))

print(f"\n📝 Diagnostico guardado en: {LOG_FILE}")


Verificacion del archivo: senales_heuristicas_2025-05-27.csv
Total filas: 5134161

Columnas requeridas: {'fecha', 'signal', 'estrategia', 'simbolo'}
Columnas presentes : {'fecha', 'signal', 'estrategia', 'simbolo'}
✅ Todas las columnas requeridas están presentes.

Tipos de columnas:
fecha         datetime64[ns]
simbolo               object
estrategia            object
signal                object
dtype: object

Valores únicos en 'signal': ['buy', 'sell']

Conteo por tipo de señal:
sell    2710909
buy     2423252
Name: signal, dtype: int64

Fechas:
Fecha mínima: 1998-06-08 00:00:00
Fecha máxima: 2025-05-23 00:00:00

Columnas con valores nulos:
Sin nulos

✅ El archivo de señales esta listo para backtesting.

📝 Diagnostico guardado en: C:\Users\leant\OneDrive\Documentos\trading\Jupyter\reports\senales_heuristicas\diagnostico_senales_2025-05-27_12-57.txt
